# ETLs

## Imports

In [120]:
import pandas as pd
import sqlite3

select_tables = "SELECT name FROM sqlite_master WHERE type='table'"

sales_con = sqlite3.connect("go_sales.sqlite")
sales_tables = pd.read_sql_query(select_tables, sales_con)

sales_country       = pd.read_sql_query("SELECT * FROM country;", sales_con)
order_details       = pd.read_sql_query("SELECT * FROM order_details;", sales_con)
order_header        = pd.read_sql_query("SELECT * FROM order_header;", sales_con)
order_method        = pd.read_sql_query("SELECT * FROM order_method;", sales_con)
product             = pd.read_sql_query("SELECT * FROM product;", sales_con)
product_line        = pd.read_sql_query("SELECT * FROM product_line;", sales_con)
product_type        = pd.read_sql_query("SELECT * FROM product_type;", sales_con)
sales_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site;", sales_con)
return_reason       = pd.read_sql_query("SELECT * FROM return_reason;", sales_con)
returned_item       = pd.read_sql_query("SELECT * FROM returned_item;", sales_con)
sales_branch        = pd.read_sql_query("SELECT * FROM sales_branch;", sales_con)
sales_staff         = pd.read_sql_query("SELECT * FROM sales_staff;", sales_con)
SALES_TARGETData    = pd.read_sql_query("SELECT * FROM SALES_TARGETData;", sales_con)
sqlite_sequence     = pd.read_sql_query("SELECT * FROM sqlite_sequence;", sales_con)
print("Imported sales tables")

staff_con = sqlite3.connect("go_staff.sqlite")
staff_tables = pd.read_sql_query(select_tables, staff_con)

course            = pd.read_sql_query("SELECT * FROM course;", staff_con)
sales_branch      = pd.read_sql_query("SELECT * FROM sales_branch;", staff_con)
sales_staff       = pd.read_sql_query("SELECT * FROM sales_staff;", staff_con)
satisfaction      = pd.read_sql_query("SELECT * FROM satisfaction;", staff_con)
satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type;", staff_con)
training          = pd.read_sql_query("SELECT * FROM training;", staff_con)
print("Imported staff tables")

crm_con = sqlite3.connect("go_crm.sqlite")
crm_tables = pd.read_sql_query(select_tables, crm_con)
                           
age_group             = pd.read_sql_query("SELECT * FROM age_group;", crm_con)
crm_country           = pd.read_sql_query("SELECT * FROM country;", crm_con)
retailer              = pd.read_sql_query("SELECT * FROM retailer;", crm_con)
retailer_contact      = pd.read_sql_query("SELECT * FROM retailer_contact;", crm_con)
retailer_headquarters = pd.read_sql_query("SELECT * FROM retailer_headquarters;", crm_con)
retailer_segment      = pd.read_sql_query("SELECT * FROM retailer_segment;", crm_con)
crm_retailer_site     = pd.read_sql_query("SELECT * FROM retailer_site;", crm_con)
retailer_type         = pd.read_sql_query("SELECT * FROM retailer_type;", crm_con)
sales_demographic     = pd.read_sql_query("SELECT * FROM sales_demographic;", crm_con)
sales_territory       = pd.read_sql_query("SELECT * FROM sales_territory;", crm_con)
print("Imported crm tables")

inventory_level = pd.read_csv("GO_SALES_INVENTORY_LEVELSData.csv")
print("Imported inventory table")

product_forecast = pd.read_csv("GO_SALES_PRODUCT_FORECASTData.csv")
print("Imported product forecast table")

Imported sales tables
Imported staff tables
Imported crm tables
Imported inventory table
Imported product forecast table


In [121]:
"""
Flexible method to merge two tables
- NaN values of one dataframe can be filled by the other dataframe
- Uses all available columns
- Errors when a row of the two dataframes doesn't match (df1 has 'A' and df2 has 'B' in row)
"""
def merge_tables(df1, df2, index_col):
    # Ensure 'CODE' is set as the index for both DataFrames
    if index_col not in df1.columns or index_col not in df2.columns:
        raise KeyError(f"{index_col} must be a column in both DataFrames.")
    
    df1 = df1.set_index(index_col)
    df2 = df2.set_index(index_col)

    # Identify common and exclusive columns
    common_columns = df1.columns.intersection(df2.columns)
    exclusive_df1 = df1.columns.difference(df2.columns)
    exclusive_df2 = df2.columns.difference(df1.columns)

    # Concatenate exclusive columns from each DataFrame onto the other
    df1_combined = pd.concat([df1, df2[exclusive_df2]], axis=1, sort=False)
    df2_combined = pd.concat([df2, df1[exclusive_df1]], axis=1, sort=False)

    # Resolve common columns with nulls and conflicts
    for col in common_columns:
        # Align the Series from both DataFrames for comparison
        series1, series2 = df1_combined[col].align(df2_combined[col])

        # Check for conflicts (non-null values that do not match)
        conflict_mask = (~series1.isnull() & ~series2.isnull() & (series1 != series2))
        if conflict_mask.any():
            raise ValueError(f"Merge failed due to conflict in column '{col}'")

        # Use values from df2 where df1 is null (prioritizing df1 values)
        df1_combined[col] = series1.combine_first(series2)

    return df1_combined

# Merge duplicate tables into single table
retailer_site = merge_tables(sales_retailer_site, crm_retailer_site, 'RETAILER_SITE_CODE')
# Column name mismatch
sales_country = sales_country.rename(columns={'COUNTRY': 'COUNTRY_EN'})
country = merge_tables(sales_country, crm_country, 'COUNTRY_CODE')

## Product ETL

In [122]:
product_etl = pd.merge(product, product_type, on="PRODUCT_TYPE_CODE")
product_etl = pd.merge(product_etl, product_line, on="PRODUCT_LINE_CODE")\
    [[
        "PRODUCT_NAME", "PRODUCT_IMAGE", "PRODUCT_NUMBER",
        "DESCRIPTION", "LANGUAGE", "PRODUCTION_COST",
        "MARGIN", "INTRODUCTION_DATE", "PRODUCT_LINE_CODE",
        "PRODUCT_LINE_EN"
    ]]\
    .rename(columns={
        "PRODUCT_NAME": "PRODUCT_name",
        "PRODUCT_IMAGE": "PRODUCT_image",
        "PRODUCT_NUMBER": "PRODUCT_id",
        "DESCRIPTION": "PRODUCT_description",
        "LANGUAGE": "PRODUCT_LANGUAGE_name",
        "PRODUCTION_COST": "PRODUCT_PRODUCTION_COST_cost",
        "MARGIN": "PRODUCT_MARGIN_percentage",
        "INTRODUCTION_DATE": "PRODUCT_INTRODUCTION_DATE_date",
        "PRODUCT_LINE_CODE": "PRODUCT_LINE_code",
        "PRODUCT_LINE_EN": "PRODUCT_LINE_name"
    })

product_etl

,PRODUCT_name,PRODUCT_image,PRODUCT_id,PRODUCT_description,PRODUCT_LANGUAGE_name,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_percentage,PRODUCT_INTRODUCTION_DATE_date,PRODUCT_LINE_code,PRODUCT_LINE_name
0,TrailChef Water Bag,P01CE1CG1.jpg,1,"Lightweight, collapsible bag to carry liquids ...",EN,4,.33,15-2-2011,1,Camping Equipment
1,TrailChef Utensils,P10CE1CG1.jpg,10,"Spoon, fork and knife set made of a light yet ...",EN,10,.4,15-2-2011,1,Camping Equipment
2,Insect Bite Relief,P100OP4FA17.jpg,100,The Insect Bite Relief helps the itching and s...,EN,3,.5,15-2-2011,4,Outdoor Protection
3,Hailstorm Steel Irons,P101GE5IR18.jpg,101,Iron is 17-4 stainless steel. Shafts are grap...,EN,305.54,.43,15-12-2019,5,Golf Equipment
4,Hailstorm Titanium Irons,P102GE5IR18.jpg,102,Made entirely of pure titanium. The ultimate i...,EN,380.95,.51,10-12-2019,5,Golf Equipment
...,...,...,...,...,...,...,...,...,...,...
110,Sun Shield,P91OP4SS16.jpg,95,"PABA free sunscreen, SPF 30, poison oak and iv...",EN,3,.5,15-2-2011,4,Outdoor Protection
111,Compact Relief Kit,P96OP4FA17.jpg,96,A personal first aid kit is recommended for ev...,EN,16.43,.28,15-2-2011,4,Outdoor Protection
112,Deluxe Family Relief Kit,P96OP4FA17.jpg,97,A complete medical kit suitable for families w...,EN,25,.28,5-3-2013,4,Outdoor Protection
113,Calamine Relief,P98OP4FA17.jpg,98,Use the Calamine Relief for allergic skin reac...,EN,3,.5,15-2-2011,4,Outdoor Protection


## Sales Staff ETL

In [123]:
sales_staff_etl = pd.merge(sales_staff, sales_branch, on='SALES_BRANCH_CODE')
sales_staff_etl = pd.merge(sales_staff_etl, country, on='COUNTRY_CODE')
sales_staff_etl = pd.merge(sales_staff_etl, sales_territory, on='SALES_TERRITORY_CODE')\
    [['SALES_STAFF_CODE', 'FIRST_NAME', 'LAST_NAME', 'POSITION_EN',
       'WORK_PHONE', 'EXTENSION', 'FAX', 'EMAIL', 'DATE_HIRED',
       'SALES_BRANCH_CODE', 'MANAGER_CODE', 'ADDRESS1',
       'ADDRESS2', 'CITY', 'REGION', 'POSTAL_ZONE', 'COUNTRY_CODE', 
       'COUNTRY_EN', 'LANGUAGE', 'CURRENCY_NAME', 
       'FLAG_IMAGE', 'SALES_TERRITORY_CODE',
       'TERRITORY_NAME_EN']]\
        .rename(columns={
            'SALES_STAFF_CODE': 'SALES_STAFF_code', 
            'FIRST_NAME': 'FIRST_NAME_name', 
            'LAST_NAME': 'LAST_NAME_name', 
            'POSITION_EN' : 'POSITION_name',
            'WORK_PHONE': 'WORK_PHONE_number', 
            'EXTENSION': 'EXTENSION_number', 
            'FAX': 'FAX_number', 
            'EMAIL': 'EMAIL_address', 
            'DATE_HIRED': 'DATE_HIRED_date',
            'SALES_BRANCH_CODE': 'SALES_BRANCH_code', 
            'MANAGER_CODE': "MANAGER_code", 
            'ADDRESS1': 'ADDRESS1_adres',
            'ADDRESS2': 'ADDRESS2_adres', 
            'CITY': 'CITY_name', 
            'REGION': 'REGION_name', 
            'POSTAL_ZONE': 'POSTAL_ZONE_code', 
            'COUNTRY_CODE': 'COUNTRY_code', 
            'COUNTRY_EN': 'COUNTRY_name', 
            'LANGUAGE' : 'LANGUAGE_name', 
            'CURRENCY_NAME': 'CURRENCY_name', 
            'FLAG_IMAGE': 'FLAG_image', 
            'SALES_TERRITORY_CODE': 'SALES_TERRITORY_code',
            'TERRITORY_NAME_EN': 'TERRITORY_name',
        })

sales_staff_etl

,SALES_STAFF_code,FIRST_NAME_name,LAST_NAME_name,POSITION_name,WORK_PHONE_number,EXTENSION_number,FAX_number,EMAIL_adress,DATE_HIRED_date,SALES_BRANCH_code,...,CITY_name,REGION_name,POSTAL_ZONE_code,COUNTRY_code,COUNTRY_name,LANGUAGE_name,CURRENCY_name,FLAG_image,SALES_TERRITORY_code,TERRITORY_name
0,100,Tuomas,Savolainen,Level 2 Sales Representative,+358(0)17 - 433 127,825,+358(0)17 - 433 129,TSavolainen@grtd123.com,23-Jul-1998 12:00:00 AM,31,...,Kuopio,None,FIN-73100,22,Finland,EN,markka,F21,5,Northern Europe
1,101,Chang-ho,Kim,Level 2 Sales Representative,82-2-778-6587,1228,82-2-778-6586,CKim@grtd123.com,15-Dec-1998 12:00:00 AM,32,...,Seoul,None,121-020,13,Korea,EN,won,F13,4,Asia Pacific
2,102,Jung-ho,Choi,Level 3 Sales Representative,82-2-778-6587,2641,82-2-778-6586,JChoi@grtd123.com,27-Nov-1998 12:00:00 AM,32,...,Seoul,None,121-020,13,Korea,EN,won,F13,4,Asia Pacific
3,103,Kwei-Ping,Kao,Level 2 Sales Representative,886-2-2975-1988,1541,886-2-2989-4978,KKwao@grtd123.com,07-Oct-1998 12:00:00 AM,33,...,San Chung,Taipei,241,12,Taiwan,EN,new dollar,F12,4,Asia Pacific
4,104,Chin-Tsai,Fang,Level 3 Sales Representative,886-2-2975-1988,2311,886-2-2989-4978,CFon@grtd123.com,08-May-1996 12:00:00 AM,33,...,San Chung,Taipei,241,12,Taiwan,EN,new dollar,F12,4,Asia Pacific
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-Feb-1998 12:00:00 AM,29,...,Bilbao,Vizcaya,48223,19,Spain,EN,pesetas,F18,7,Southern Europe
98,96,Bayard,Lopes,Branch Manager,+55 (11) 344-4444,3490,+55 (11) 333-2223,BLopes@grtd123.com,12-Aug-1997 12:00:00 AM,30,...,São Paulo,SP,01403-090,21,Brazil,EN,reals,F20,1,Americas
99,97,Viviam,Barros,Level 3 Sales Representative,+55 (11) 344-4444,3364,+55 (11) 333-2223,VBarros@grtd123.com,10-Feb-2001 12:00:00 AM,30,...,São Paulo,SP,01403-090,21,Brazil,EN,reals,F20,1,Americas
100,98,Eduardo,Guimarães,Level 2 Sales Representative,+55 (11) 344-4444,1442,+55 (11) 333-2223,EGuimaraes@grtd123.com,14-Feb-1999 12:00:00 AM,30,...,São Paulo,SP,01403-090,21,Brazil,EN,reals,F20,1,Americas


## Retailer Contact ETL

In [131]:
retailer_contact_etl = pd.merge(retailer_contact, retailer_site, on='RETAILER_SITE_CODE')
retailer_contact_etl = pd.merge(retailer_contact_etl, country, on='COUNTRY_CODE')
retailer_contact_etl = pd.merge(retailer_contact_etl, sales_territory, on='SALES_TERRITORY_CODE')\
    [['RETAILER_CONTACT_CODE', 'RETAILER_SITE_CODE', 'FIRST_NAME',
       'LAST_NAME', 'JOB_POSITION_EN', 'EXTENSION', 'FAX', 'E_MAIL', 'GENDER', 
       'RETAILER_CODE', 'ADDRESS1', 'ADDRESS2', 'CITY', 'REGION',
       'POSTAL_ZONE', 'COUNTRY_CODE', 'ACTIVE_INDICATOR', 
       'COUNTRY_EN', 'LANGUAGE', 'CURRENCY_NAME', 
       'FLAG_IMAGE', 'SALES_TERRITORY_CODE',
       'TERRITORY_NAME_EN']]\
    .rename(columns={
        'RETAILER_CONTACT_CODE' : 'RETAILER_CONTACT_code', 
        'RETAILER_SITE_CODE': 'RETAILER_SITE_code', 
        'FIRST_NAME' : 'FIRST_NAME_name',
        'LAST_NAME': 'LAST_NAME_name', 
        'JOB_POSITION_EN' : 'JOB_POSITION_EN',
        'EXTENSION': 'EXTENSION_number', 
        'FAX': 'FAX_number', 
        'E_MAIL': 'EMAIL_address', 
        'GENDER': 'GENDER_code', 
        'RETAILER_CODE': 'RETAILER_code', 
        'ADDRESS1': 'ADDRESS1_address', 
        'ADDRESS2': 'ADDRESS1_address', 
        'CITY': 'CITY_name', 
        'REGION': 'REGION_name',
        'POSTAL_ZONE': 'POSTAL_ZONE_code', 
        'COUNTRY_CODE': 'COUNTRY_code', 
        'ACTIVE_INDICATOR': 'ACTIVE_INDICATOR_bool',
        'COUNTRY_EN': 'COUNTRY_name', 
        'LANGUAGE' : 'LANGUAGE_name', 
        'CURRENCY_NAME': 'CURRENCY_name', 
        'FLAG_IMAGE': 'FLAG_image', 
        'SALES_TERRITORY_CODE': 'SALES_TERRITORY_code',
        'TERRITORY_NAME_EN': 'TERRITORY_name',
    })

retailer_contact_etl

,RETAILER_CONTACT_code,RETAILER_SITE_code,FIRST_NAME_name,LAST_NAME_name,JOB_POSITION_EN,EXTENSION_number,FAX_number,EMAIL_address,GENDER_code,RETAILER_code,...,REGION_name,POSTAL_ZONE_code,COUNTRY_code,ACTIVE_INDICATOR_bool,COUNTRY_name,LANGUAGE_name,CURRENCY_name,FLAG_image,SALES_TERRITORY_code,TERRITORY_name
0,10,14,Frank,Smith,Chief Purchaser,2489,1 (514) 741-1610,FSmith@legolfeurinc.com,M,93,...,Québec,H2Y 2W2,4,1,Canada,EN,dollars,F04,1,Americas
1,100,85,Amanda,Yates,Stock Manager,None,1 (702) 369-2274,Y2883@emertxe.com,F,104,...,Nevada,89118,3,1,United States,EN,dollars,F03,1,Americas
2,101,84,Paula,Schmidt,Assistant Purchaser,None,1 (702) 882-0978,S8832@emertxe.com,F,104,...,Nevada,89763,3,1,United States,EN,dollars,F03,1,Americas
3,102,88,Micheal,Tao,District Manager,445,1 (713) 524-3215,T8839@emertxe.com,M,104,...,Texas,77112,3,1,United States,EN,dollars,F03,1,Americas
4,103,92,Jack,Murphy,District Manager,None,1 (305) 557-5478,Murphy@eyedimensions3.com,M,105,...,Florida,33021,3,0,United States,EN,dollars,F03,1,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,95,87,Jennifer,Gomes,Site Manager,None,1 (423) 266-4721,G1289@emertxe.com,F,104,...,Tennessee,37405-0955,3,1,United States,EN,dollars,F03,1,Americas
387,96,91,Timothy,Moon,Site Assistant Manager,None,1 (304) 343-4560,M0019@emertxe.com,M,104,...,West Virginia,25301-1299,3,1,United States,EN,dollars,F03,1,Americas
388,97,90,Paul,Fischer,Stock Manager,None,1 (206) 362-6645,F9943@emertxe.com,M,104,...,Washington,98154,3,1,United States,EN,dollars,F03,1,Americas
389,98,89,Derek,Wilkes,Site Assistant Manager,None,1 (703) 549-4896,W3219@emertxe.com,M,104,...,Virginia,22323,3,1,United States,EN,dollars,F03,1,Americas
